In [ ]:
'''
Name                  : xxGmail
Description           : This code is to connect the Gmail and fetch some banking transaction emails.
                        The subject of this email is hardcoded.
Author                : Prabu M
Date                  : 03/02/2018
Contact               : mprabu.m@gmail.com/+65-81503178
'''
import smtplib
import datetime
import time
import imaplib
import email
import re
import string
from __future__ import print_function
from beautifultable import BeautifulTable

class Gmail(object):
    def __init__(self, FROM_EMAIL, FROM_PWD):
        self.FROM_EMAIL = FROM_EMAIL
        self.FROM_PWD   = FROM_PWD
        
    def Display(self,row1):
        try:
            table = BeautifulTable()
            self.row1 = row1
            table.column_headers = ["Date", "Payee", "Value"]
            for x in self.row1:
                table.append_row(self.row1[x])
            print(table)
        except:
            print("ERROR:display error.")

    # Connect Gmail and read the mail!!
    def read_email_from_gmail(self):
        ORG_EMAIL   = "@gmail.com"
        SMTP_SERVER = "imap.gmail.com"
        SMTP_PORT   = 993
        row = {}
        
        try:
            sub = "Transaction alert for your ICICI Bank Internet Banking"
            mail = imaplib.IMAP4_SSL(SMTP_SERVER)
            mail.login(self.FROM_EMAIL,self.FROM_PWD)
            mail.select('inbox')

            type, data = mail.search(None, 'ALL')
            mail_ids = data[0]

            id_list = mail_ids.split()   
            first_email_id = int(id_list[0])
            latest_email_id = int(id_list[-1])
            j = 0
            for i in range(latest_email_id,first_email_id, -1):
                typ, data = mail.fetch(i, '(RFC822)' )
                j += 1
                tr_date = []
                for response_part in data:
                    
                    if isinstance(response_part, tuple):
                        msg = email.message_from_string(response_part[1])                    
                        email_subject = msg['subject']
                        email_from = msg['from']
                        if sub in email_subject:
                            if msg.is_multipart():
                                for payload in msg.walk():
                                    line = payload.get_payload()
                                    regex = r">You have made an.*"                                
                                    matches = re.finditer(regex, str(line))
                                    for matchNum, match in enumerate(matches):
                                        matchNum = matchNum + 1
                                        lst = match.group().split()
                                        conv=time.strptime(string.replace(str((lst[lst.index('on')+1])),","," "),"%b %d %Y")
                                        tr_date.append(time.strftime("%d/%m/%Y",conv))
                                        tr_date.append(lst[lst.index('payee')+1])
                                        tr_date.append(lst[lst.index('INR')+1])
                                        row[j]= tr_date
                            else:
                                print(msg.get_payload())
            b.Display(row)       
        except:
            print("ERROR:Gmail is not connected. Please check the secure access in your gmail")
                
# Call the function        
FROM_EMAIL = raw_input("Enter your Gmail user name:")
FROM_PWD   = raw_input("Enter your Gmail Password:")
b = Gmail(FROM_EMAIL,FROM_PWD)

b.read_email_from_gmail()